# Code for calculating the necessary metrics for pitch control and PCV in sliding time windows of 2. The average values for PC and PCV in the 2-minute time windows represent the flow of momentum.

In [ ]:
#two minute calculations
import os
from progressbar import ProgressBar
import pandas as pd
import pickle
path = '/Users/gebruiker/Dropbox/Master Thesis/Master thesis RKC Waalwijk/Code/Pitch control per minute'
home_files = []
away_files = []
#Create a list of all 100 files used for pitch control
for file in os.listdir(path):
    if 'dfhome' in file:
        home_files.append(file)
    elif 'dfaway' in file:
        away_files.append(file)
home_files = sorted(home_files)
away_files = sorted(away_files)
#iterate over each file
for file in range(len(home_files)):
    #import home and away tracking dataframes
    df_home = pd.read_excel(home_files[file])
    df_away = pd.read_excel(away_files[file])
    df_home = df_home.set_index("Frame_number")
    df_away = df_away.set_index("Frame_number")
    
    #only do the calculations when the ball is in play
    df_home_alive = df_home.loc[df_home['Ball_status'] == 'Alive']
    df_away_alive = df_away.loc[df_away['Ball_status'] == 'Alive']
    df_home_alive = df_home_alive.loc[df_home_alive.PPCF.map(type)==float]
    df_away_alive = df_away_alive.loc[df_away_alive.PPCF.map(type)==float]
    
    
    #initialize lists for pitch control and PCV for the home and away team for the 2 minute time windows
    meanpchome = []
    meanpcaway = []
    meanpcvhome = []
    meanpcvaway = []
    pchome = []
    pcaway = []
    pcvhome = []
    pcvaway = []
    Time = []
    half = []    
    pbar = ProgressBar()
    
    #iterate over the length of the dataframe
    for i in pbar(range(len(df_home_alive))):
        half.append(df_home_alive.iloc[i]['Half'])
        
        #only select the two-minute time windows for each row
        df_i_home = df_home_alive.loc[df_home_alive.index[df_home_alive.index >= df_home_alive.iloc[max(0, i - 60*25*2)].name].min(): df_home_alive.iloc[i].name]
        df_i_away = df_away_alive.loc[df_away_alive.index[df_away_alive.index >= df_away_alive.iloc[max(0, i - 60*25*2)].name].min(): df_away_alive.iloc[i].name]                                  
        
        #add the average PC an PCV values in the time windows, and the PC and PCV values of that moment to a list.
        #This will be later used to calculate the flow of momentum and the necessary other variables
        meanpchome.append(df_i_home['PPCF'].mean())
        meanpcaway.append(1-df_i_home['PPCF'].mean())
        meanpcvhome.append(df_i_home['Weighted_PPCF'].mean())
        meanpcvaway.append(df_i_away['Weighted_PPCF'].mean())
        pchome.append(df_home_alive.iloc[i]['PPCF'])
        pcaway.append(1-df_home_alive.iloc[i]['PPCF'])
        pcvhome.append(df_home_alive.iloc[i]['Weighted_PPCF'])
        pcvaway.append(df_away_alive.iloc[i]['Weighted_PPCF'])
        Time.append(df_home_alive.iloc[i]['Time_in_game']/60)

    data = {'momentum pc home previous 2 minutes': meanpchome2,
            'momentum pc away previous 2 minutes': meanpcaway2,
            'momentum pcv home previous 2 minutes': meanpcvhome2,
            'momentum pcv away previous 2 minutes': meanpcvaway2,
            'pc home': pchome,
            'pc away': pcaway,
            'pcv home': pcvhome,
            'pcv away': pcvaway,
            'Half': half,
            'Time': Time}

    #create pickle file for each match in the data for pitch control and PCV
    df = pd.DataFrame(data)
    pickle.dump(df, open('pc_over_last_two_minutes_final' + str(home_files[file][7:14]) + '.p', 'wb'))  
    
        